In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [70]:
# Data format: inputs: [[x1, x2], [x1, x2], ...], targets: [y, y, ...]

max_num = 31
data_size = 512

inputs, targets = [], []
import random
for i in range(data_size):
  x1 = random.randint(0, max_num)
  x2 = random.randint(0, max_num)
  inputs.append([x1, x2])
  targets.append(x1 + x2)
  
inputs_t, targets_t = torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

dataset = torch.utils.data.TensorDataset(inputs_t, targets_t)
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

print(len(train_dataset))
print(len(test_dataset))
print(train_dataset[0])



409
103
(tensor([ 1., 23.]), tensor(24.))


In [17]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cpu


In [71]:
torch.manual_seed(1)
batch_size = 32


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, **kwargs)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cpu


In [67]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        # self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(4, 1)

    def forward(self, x):
        # x = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x)))))
        x = self.fc3(F.sigmoid(self.fc1(x)))
        return x.squeeze()


model = Net().to(device)
summary(model, input_size=(1, 2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 4]              12
            Linear-2                 [-1, 1, 1]               5
Total params: 17
Trainable params: 17
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [68]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)          # Use MSE loss instead of nll
        loss.backward()
        optimizer.step()

        # pred = output.argmax(dim=0, keepdim=True)  # get the index of the max log-probability
        # print(data.shape)
        # print(target.shape)
        # print(output.shape)
        # print(pred.shape)
        # correct = pred.eq(target.view_as(pred)).sum().item()

        # pbar.set_description(desc= f'EPOCH: {epoch}, loss={loss.item()} batch_id={batch_idx}, Accuracy: {correct / len(pred) * 100.}')
        correct = sum(abs(torch.round(target) - torch.round(output)) == 0)
        pbar.set_description(desc= f'EPOCH: {epoch}, loss={loss.item()} batch_id={batch_idx}, Accuracy: {correct / len(output) * 100.}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item()  # sum up batch loss
            # pred = output.argmax(dim=0, keepdim=True)  # get the index of the max log-probability
            # correct += pred.eq(target.view_as(pred)).sum().item()
            correct += sum(abs(torch.round(target) - torch.round(output)) == 0)
            print(f'target: {torch.round(target)}, output: {torch.round(output)}')

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [56]:
a = torch.tensor([1., 2., 3.])
b = torch.tensor([1.7, 0, 2.8])

print(sum(abs(torch.round(a) - torch.round(b)) == 0))

tensor(1)


In [72]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 50
for epoch in range(1, num_epochs):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 1, loss=154.94520568847656 batch_id=12, Accuracy: 4.0: 100%|██████████| 13/13 [00:00<00:00, 203.28it/s]


target: tensor([17., 32., 31.,  1., 16., 46., 40., 24., 43., 30., 24., 15., 10., 29.,
        14., 23., 29., 48., 36., 20., 32., 22., 30., 40., 50., 33., 42., 28.,
        45.,  9., 34., 26.]), output: tensor([30., 30., 32., 30., 32., 30., 32., 30., 32., 32., 30., 32., 30., 30.,
        32., 31., 32., 30., 32., 30., 30., 30., 30., 30., 32., 32., 31., 30.,
        32., 32., 32., 32.])
target: tensor([58., 32., 16., 40., 58., 26., 32.,  8., 42., 34., 41., 31., 13., 14.,
        40., 47., 36., 46., 28., 32., 45., 22., 27., 25., 40., 43., 44., 24.,
         9., 33., 29., 39.]), output: tensor([30., 30., 30., 32., 31., 30., 32., 32., 31., 32., 32., 30., 30., 30.,
        32., 32., 32., 30., 30., 30., 31., 30., 30., 30., 30., 30., 31., 30.,
        30., 32., 30., 32.])
target: tensor([30., 37., 24., 31., 27., 19., 18., 18., 57., 23., 41., 38., 17., 34.,
        14., 25., 31., 59., 36., 10., 23., 33.,  7., 32., 50., 26., 26., 59.,
        36., 44., 35., 37.]), output: tensor([30., 30., 32., 3

EPOCH: 2, loss=138.1938934326172 batch_id=12, Accuracy: 8.0: 100%|██████████| 13/13 [00:00<00:00, 198.89it/s]


target: tensor([ 8., 40., 25., 23., 17., 33., 41., 18., 45., 26., 18., 35., 37., 44.,
        32., 42., 37., 23., 13., 32., 22., 44., 18., 26., 43., 46., 59., 47.,
        30.,  9., 18., 32.]), output: tensor([31., 31., 31., 31., 28., 31., 31., 31., 31., 31., 28., 28., 31., 31.,
        31., 31., 31., 31., 28., 28., 31., 31., 31., 28., 31., 31., 31., 31.,
        31., 28., 31., 31.])
target: tensor([17., 38., 10., 28., 58., 31.,  8., 34., 26., 41., 46., 40., 32., 14.,
        45., 36., 59., 58., 46., 27., 31., 16., 24., 60., 14., 30., 19., 40.,
        50.,  9., 24., 31.]), output: tensor([28., 31., 31., 28., 31., 31., 31., 31., 31., 31., 30., 31., 29., 31.,
        31., 29., 31., 31., 31., 31., 31., 29., 31., 31., 31., 28., 31., 31.,
        31., 31., 31., 31.])
target: tensor([34., 26., 34., 32., 29., 33., 10., 27., 42., 36., 20., 30.,  1., 22.,
        48., 25., 15., 28., 14., 36., 16., 24., 29., 24., 23., 50., 35., 32.,
        57., 40., 31.,  7.]), output: tensor([31., 28., 31., 2

EPOCH: 3, loss=203.94384765625 batch_id=12, Accuracy: 4.0: 100%|██████████| 13/13 [00:00<00:00, 218.25it/s]


...

target: tensor([45., 44., 46., 22., 58., 31., 17., 46., 32., 27.,  9., 31., 28., 32.,
        28., 18.,  1., 14., 24., 36.,  9., 42., 38., 33., 17., 42., 31., 60.,
        37., 18., 34.,  7.]), output: tensor([34., 34., 34., 32., 34., 34., 34., 34., 19., 22., 19., 34., 34., 34.,
        19., 22., 19., 22., 34., 34., 22., 34., 34., 34., 19., 34., 22., 34.,
        34., 34., 34., 19.])
target: tensor([29., 29., 22., 50., 36., 25., 30.]), output: tensor([22., 34., 34., 34., 34., 22., 19.])

Test set: Average loss: 123.0262, Accuracy: 3/103 (3%)



EPOCH: 48, loss=110.47671508789062 batch_id=12, Accuracy: 4.0: 100%|██████████| 13/13 [00:00<00:00, 160.21it/s]


target: tensor([44., 26., 29., 23., 17., 23., 31., 37., 14., 50., 47., 40., 31., 16.,
        20., 32., 36., 41., 26., 27., 37., 18., 26., 32., 18., 41., 35., 46.,
        28., 10., 33., 32.]), output: tensor([37., 37., 37., 37., 25., 25., 25., 37., 21., 37., 37., 25., 37., 25.,
        37., 21., 25., 37., 21., 37., 37., 37., 25., 37., 25., 25., 37., 37.,
        21., 21., 25., 21.])
target: tensor([27., 32., 59., 22.,  8., 36., 44., 31., 38., 19., 16., 50., 22., 40.,
        60., 15., 14., 29.,  9.,  9., 36.,  8.,  7., 40., 26., 39., 45., 42.,
        36., 23., 24., 30.]), output: tensor([25., 37., 37., 21., 25., 27., 37., 25., 37., 37., 32., 37., 37., 37.,
        37., 25., 25., 21., 21., 25., 37., 25., 21., 25., 21., 37., 37., 37.,
        25., 25., 25., 21.])
target: tensor([46., 18., 14., 45., 25., 42., 10., 34., 17., 35., 47., 46., 40., 32.,
        13., 59., 30., 33., 24., 40., 18., 25., 34., 34., 28., 29., 43., 57.,
        33., 58., 32., 24.]), output: tensor([37., 21., 25., 3

EPOCH: 49, loss=214.3831024169922 batch_id=12, Accuracy: 4.0: 100%|██████████| 13/13 [00:00<00:00, 127.03it/s]


target: tensor([46., 40., 31., 22., 24., 28., 44., 36., 59., 23.,  8., 45., 41., 22.,
        30., 40., 37., 60., 41., 35., 24., 42.,  8., 59., 58., 18., 14., 47.,
        25.,  9., 27., 23.]), output: tensor([27., 27., 31., 27., 27., 27., 27., 31., 40., 31., 31., 42., 27., 27.,
        27., 31., 27., 27., 31., 27., 27., 39., 25., 42., 42., 16., 31., 31.,
        27., 27., 31., 31.])
target: tensor([43., 50., 46., 26., 58., 32., 23., 13., 17., 15., 36., 29., 44., 39.,
        26., 10., 24., 35., 18., 34.,  1., 32., 29., 36.,  9., 47., 26., 34.,
        30., 32., 28., 31.]), output: tensor([31., 31., 27., 27., 27., 27., 27., 27., 27., 31., 31., 27., 27., 31.,
        31., 27., 27., 27., 31., 31., 27., 31., 27., 27., 31., 27., 27., 31.,
        31., 27., 27., 31.])
target: tensor([33., 37., 25., 31., 33., 29., 31., 17., 33., 34., 18., 16., 57., 40.,
        19., 14., 40., 26., 27., 32., 36., 20., 32., 46., 38.,  7., 43., 48.,
        45., 10., 14., 50.]), output: tensor([31., 27., 31., 3